<a href="https://colab.research.google.com/github/Mattrfu/speechwriting-LLM/blob/main/Speechwriting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install accelerate
!pip install trl
!pip install jsonlines
!pip install einops


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# from transformers import AutoTokenizer
# import transformers
# import torch

# model = "meta-llama/Llama-2-7b-hf"

# tokenizer = AutoTokenizer.from_pretrained(model)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# text = "Write a speech in the style of David Hogg about waffles"
# sequences = pipeline(
#     text,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=1000,
# )
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map="auto")

In [ ]:
# !export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:16'

In [ ]:
# text = "Write a speech in the style of David Hogg about waffles"
# inputs = tokenizer(text, return_tensors = "pt")
# outputs = model(**inputs)
# print(inputs)
# print(outputs)

In [ ]:
# !curl https://huggingface.co/datasets/clam004/antihallucination_dataset/resolve/main/antihallucination.jsonl -o antiha llucination.jsonl


In [ ]:
# inputs

In [ ]:
# inputs["input_ids"].shape

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
# text = "<Prompt>: Write a short length speech for a national audience in the style of David Hogg on the topic of gun control. <Bot>:First off, I’m gonna start off by putting this price tag right here as a reminder for you guys to know how much Marco Rubio took for every student’s life in Florida. One dollar and five cents. The cold grasp of corruption shackles the District of Columbia. The winter is over. Change is here. The sun shines on a new day, and the day is ours. First-time voters show up 18 percent of the time at midterm elections. Not anymore. Now, who here is gonna vote in the 2018 election? If you listen real close, you can hear the people in power shaking. They’ve gotten used to being protective of their position, the safety of inaction. Inaction is no longer safe. And to that, we say: No more. Ninety-six people die every day from guns in our country, yet most representatives have no public stance on guns. And to that, we say: No more. We are going to make this the voting issue. We are going to take this to every election, to every state, in every city. We are going to make sure the best people get in our elections to run, not as politicians, but as Americans. Because this — this is not cutting it. When people try to suppress your vote, and there are people who stand against you because you’re too young, we say: No more. When politicians say that your voice doesn’t matter because the NRA owns them, we say: No more. When politicians send their thoughts and prayers with no action, we say: No more. And to those politicians supported by the NRA, that allow the continued slaughter of our children and our future, I say: Get your résumés ready. Today is the beginning of spring, and tomorrow is the beginning of democracy. Now is the time to come together, not as Democrats, not as Republicans, but as Americans. Americans of the same flesh and blood, that care about one thing and one thing only, and that’s the future of this country and the children that are going to lead it. Now, they will try to separate us in demographics. They will try to separate us by religion, race, congressional district and class. They will fail. We will come together. We will get rid of these public servants that only serve the gun lobby, and we will save lives. You are those heroes. Lastly, let’s put the USA over the NRA. This is the start of the spring and the blossoming of our democracy. So let’s take this to our local legislators, and let’s take this to midterm elections, because without the persistence — heat — without the persistence of voters and Americans everywhere, getting out to every election, democracy will not flourish. But it can, and it will. So, I say to those politicians that say change will not come, I say: We will not stop until every man, every woman, every child, and every American can live without fear of gun violence. And to that, I say: No more. Thank you, I love you all, God bless all of you, and God bless America. We can, and we will, change the world."
# inputs = tokenizer(text, return_tensors = "pt")

In [ ]:
# from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

In [ ]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")

In [ ]:
# !accelerate estimate-memory mistralai/Mistral-7B-v0.1

In [ ]:
# !accelerate estimate-memory stabilityai/stablelm-3b-4e1t

In [ ]:
# !accelerate estimate-memory meta-llama/Llama-2-7b-hf

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("GreenBitAI/LLaMA-2-1.1B-2bit-groupsize32", device_map=device, torch_dtype=torch.float16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("GreenBitAI/LLaMA-2-1.1B-2bit-groupsize32")

In [ ]:
import jsonlines

with jsonlines.open('output.jsonl', 'w') as writer:
    writer.write_all(dataset)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="output.jsonl")
dataset["train"].shape

In [ ]:
from transformers import AutoTokenizer

def preprocess_function(example):
    return tokenizer(example["text"], truncation = True)

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
)
tokenized_dataset

In [ ]:
len(tokenized_dataset["train"][0]["input_ids"])

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    fp16=True,
    output_dir="green_speech_model",
    evaluation_strategy="no",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    do_eval=False, # Should delete
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
text = "<Prompt>: Write a short length speech for a national audience in the style of David Hogg on the topic of waffles. <Bot>:"
inputs = tokenizer(text, return_tensors = "pt")
outputs = model.generate(**inputs, max_new_tokens=300, do_sample=True, top_k=50, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
text = "<Prompt>: Write a short length speech for a national audience in the style of David Hogg on the topic of waffles. <Bot>:"
new_model = AutoModelForCausalLM.from_pretrained("new_model")
inputs = tokenizer(text, return_tensors = "pt")
outputs = new_model.generate(**inputs, max_new_tokens=300, do_sample=True, top_k=50, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=True)